In [1]:
import numpy as np
import pandas as pd

import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from bs4 import BeautifulSoup
import requests

import time, os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from fake_useragent import UserAgent

import json

import pickle

In [2]:
# Use random user agent just in case the site checks for too many requests from a specific user agent

ua = UserAgent()
user_agent = {'User-agent': ua.random}

In [3]:
# instantiate a chrome options object so you can set the size and headless preference

chrome_options = Options()
chrome_options.add_argument(f'user-agent={user_agent}')

# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--window-size=1920x1080")

In [ ]:
# chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe" # path to the chromedriver executable
# os.environ["webdriver.chrome.driver"] = chromedriver

### Scrape main page of BGG to get name, ID, and URL of top 1000 games

In [4]:
# BGG URL of board games by ranking
bgg_top = "https://boardgamegeek.com/browse/boardgame/page/{}"


In [5]:
def get_data(start_url):
    
    """
    Takes bgg top games with page={} at the end
    and scrapes URLs.
    
    ---
    Input: bgg search page URL with page={} at the end
    Output: dictionary of game URLs ('game name':url), dataframe with columns: ['id', 'game', 'url']
    
    """
    
    page_number = 1
    links = {}
    
#     test_dict = {}
    
    id_list = []
    game_list = []
    url_list = []
    regexp = re.compile(r'boardgame/(\d+)/')
    

    while True:
        url = start_url.format(page_number)
        response  = requests.get(url, headers = user_agent)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        project_divs = soup.findAll('div', {"id":re.compile('results_objectname.*')})
                
        for div in project_divs:
            links[div.find('a').contents[0]] = "https://boardgamegeek.com" + (div.find('a').get('href'))
          
            id_list.append(int(re.search(regexp, div.find('a').get('href')).group(1)))
            game_list.append(div.find('a').contents[0])
            url_list.append("https://boardgamegeek.com" + (div.find('a').get('href')))
            
            
        test_col = ['id', 'game', 'url']
        test_df = pd.DataFrame({'id': id_list, 'game': game_list, 'url':url_list}, columns = test_col)

        if len(project_divs) == 0:
            break

        page_number +=1
        time.sleep(5)
        
        ###############
        ### For Testing
#         if page_number > 10:
#             break
        ###############
    
#     return pd.DataFrame(all_data)
                                            
    return links, test_df

In [53]:
top_1000_links2, top_1000_df2 = get_data(bgg_top)

In [54]:
top_1000_df2.head()

,id,game,url
0,174430,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...
1,161936,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...
2,167791,Terraforming Mars,https://boardgamegeek.com/boardgame/167791/ter...
3,224517,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...
4,182028,Through the Ages: A New Story of Civilization,https://boardgamegeek.com/boardgame/182028/thr...


In [55]:
top_1000_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
id      1000 non-null object
game    1000 non-null object
url     1000 non-null object
dtypes: object(3)
memory usage: 23.6+ KB


In [56]:
top_1000_df2.to_csv('df2_top_1000_1442_12May.csv', index=False)

### Given df output from 'get_data' function, scrape game description for each game in input dataframe

In [6]:
def get_game_data(my_df):
    
    """
    Returns information from each game for each URL that is in the input dataframe. 
    
    Input: dataframe in the form generated by get_data
    Output: Original dataframe with additional column: 'description'
    
    """    

    my_urls = my_df['url']
    
    description = []
           
        
    for i,url in enumerate(my_urls):
#         print(url)
        
        response  = requests.get(url, headers = user_agent)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        
        
        description.append(soup.find('meta', attrs={'name':"description"})['content'])

        time.sleep(3)
        
#     my_df['description'] = description
    my_df.insert(3, "description", description, True)
        
        
    return my_df

In [135]:
df4 = get_game_data(test_df_2)

In [136]:
df4.head()

,id,game,url,description
0,174430,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,Gloomhaven is a game of Euro-inspired tactica...
1,161936,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,Pandemic Legacy is a co-operative campaign gam...
2,167791,Terraforming Mars,https://boardgamegeek.com/boardgame/167791/ter...,"In the 2400s, mankind begins to terraform the ..."
3,224517,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,Brass: Birmingham is an economic strategy game...
4,182028,Through the Ages: A New Story of Civilization,https://boardgamegeek.com/boardgame/182028/thr...,Through the Ages: A New Story of Civilization ...


In [137]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
id             1000 non-null int64
game           1000 non-null object
url            1000 non-null object
description    1000 non-null object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [138]:
df4.tail()

,id,game,url,description
995,265402,In the Hall of the Mountain King,https://boardgamegeek.com/boardgame/265402/hal...,Play as trolls rebuilding your abandoned kingd...
996,61692,Axis & Allies Europe 1940,https://boardgamegeek.com/boardgame/61692/axis...,"Axis &amp; Allies Europe 1940, a deluxe editio..."
997,180040,Villages of Valeria,https://boardgamegeek.com/boardgame/180040/vil...,A village-building card game set in the world ...
998,176371,Explorers of the North Sea,https://boardgamegeek.com/boardgame/176371/exp...,Explorers of the North Sea is set in the latte...
999,2338,Starship Catan,https://boardgamegeek.com/boardgame/2338/stars...,Starship Catan is a two-player card game that ...


In [139]:
df4.to_csv('top_1000_desc_2230_12May.csv', index=False)

In [7]:
df4 = pd.read_csv('top_1000_desc_2230_12May.csv')

In [8]:
df4.head()

,id,game,url,description
0,174430,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,Gloomhaven is a game of Euro-inspired tactica...
1,161936,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,Pandemic Legacy is a co-operative campaign gam...
2,167791,Terraforming Mars,https://boardgamegeek.com/boardgame/167791/ter...,"In the 2400s, mankind begins to terraform the ..."
3,224517,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,Brass: Birmingham is an economic strategy game...
4,182028,Through the Ages: A New Story of Civilization,https://boardgamegeek.com/boardgame/182028/thr...,Through the Ages: A New Story of Civilization ...


In [9]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
id             1000 non-null int64
game           1000 non-null object
url            1000 non-null object
description    1000 non-null object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [10]:
df_missing = pd.read_csv('df_missing_1212_19May.csv')

In [11]:
df_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 2 columns):
id      49 non-null int64
name    49 non-null object
dtypes: int64(1), object(1)
memory usage: 912.0+ bytes


In [13]:
df_subset = pd.merge(df_missing, df4, how='left', on='id')

In [14]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Data columns (total 5 columns):
id             49 non-null int64
name           49 non-null object
game           49 non-null object
url            49 non-null object
description    49 non-null object
dtypes: int64(1), object(4)
memory usage: 2.3+ KB


In [15]:
df_subset.head()

,id,name,game,url,description
0,205637,Arkham Horror: The Card Game,Arkham Horror: The Card Game,https://boardgamegeek.com/boardgame/205637/ark...,"Something evil stirs in Arkham, and only you c..."
1,25613,Through the Ages: A Story of Civilization,Through the Ages: A Story of Civilization,https://boardgamegeek.com/boardgame/25613/thro...,Through the Ages is a civilization building ga...
2,247763,Underwater Cities,Underwater Cities,https://boardgamegeek.com/boardgame/247763/und...,"In Underwater Cities, which takes about 30-45 ..."
3,132531,Roll for the Galaxy,Roll for the Galaxy,https://boardgamegeek.com/boardgame/132531/rol...,Game description from the publisher:\n\nRoll f...
4,233371,Clank! In! Space!,Clank! In! Space!,https://boardgamegeek.com/boardgame/233371/cla...,The evil Lord Eradikus has all but conquered t...


### Given dataframe output from 'get_game_data', grab top 5 reviews for each game


In [16]:
chrome_options = Options()
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--headless")

chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

# driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver)
# driver = webdriver.Chrome(chromedriver)

In [17]:
def get_review_data(my_df):
    
    """
    Attempts to return text from text reviews of each board game
    
    Input: dataframe in the form generated by get_game_data
    Output: dataframe with additional column containing review text data
    
    """
    
    
    urls = my_df['url']
    
    reviews = []
    
#     driver = webdriver.Chrome(chromedriver)
    driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver)
    
    
        
    for url in urls:
        game_reviews=[]
#         print(url)
#         print('\n')
        driver.get(url+'/forums/63?pageid=1&sort=hot')
        time.sleep(3)
        
        soup_level1=BeautifulSoup(driver.page_source, 'lxml')
        divs = soup_level1.findAll('div', {'class': 'summary-item-callout'})
        
        
        for i in range(min(len(divs), 5)):
            review_text = ''
            new_url = 'https://boardgamegeek.com' + divs[i].find('a')['href']
            print(new_url)
            driver.get(new_url)
            time.sleep(3)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            first_post = soup2.find('div', {'class':'post-body'})
            if first_post is None:
                time.sleep(3)
                print('first_post is none')
            else:
                marks = first_post.findAll('gg-markup-safe-html')
                for mark in marks:
                    review_text = review_text + ' ' + mark.text
                game_reviews.append(review_text)
            
        reviews.append(game_reviews)
                    
    
        
        
    my_df.insert(4, "reviews", reviews, True)
        
               
    driver.close()
        
    return my_df

In [18]:
df_found = get_review_data(df_subset)

https://boardgamegeek.com/thread/1653902/capsule-reviews-arkham-horror-card-game
https://boardgamegeek.com/thread/1965630/why-you-shouldnt-go-down-rabbit-hole
https://boardgamegeek.com/thread/2172878/let-me-tell-you-about-best-game-ever
https://boardgamegeek.com/thread/1674060/unholy-matrimony-between-lcg-light-rpg-arkham-horr
https://boardgamegeek.com/thread/2093583/ah-tcg-2-years-later
https://boardgamegeek.com/thread/386394/enders-comprehensive-pictorial-overview-quintessen
https://boardgamegeek.com/thread/217249/through-ages-review
https://boardgamegeek.com/thread/511863/video-review-inevitable-march-time-three-hour-boar
https://boardgamegeek.com/thread/251655/through-ages-analysis-and-review
https://boardgamegeek.com/thread/332414/review-through-ages
https://boardgamegeek.com/thread/2207290/seaweed-always-greener-when-playing-matching-card
https://boardgamegeek.com/thread/2103596/game-terraforming-mars-wanted-be
https://boardgamegeek.com/thread/2115283/underwater-cities-light-bite

https://boardgamegeek.com/thread/484233/younger-perspective-video-review
https://boardgamegeek.com/thread/868137/you-are-entering-vicinity-area-adjacent-location
https://boardgamegeek.com/thread/1302638/guided-totally-100-safe-tour-house-hill
https://boardgamegeek.com/thread/314449/ready-some-horror-course-you-are-review
https://boardgamegeek.com/thread/719352/year-betrayal-house-hill
https://boardgamegeek.com/thread/676751/betrayal-house-hill-vs-mansions-madness
https://boardgamegeek.com/thread/119346/only-under-rated-game-geek
https://boardgamegeek.com/thread/370252/medici-when-it-fun
https://boardgamegeek.com/thread/1610942/medici-component-and-presentation-review-new-grail
https://boardgamegeek.com/thread/1677522/third-times-charm-islaythedragon-review
https://boardgamegeek.com/thread/129020/comparing-medici-and-ra-one-groups-opinion
https://boardgamegeek.com/thread/1732619/gameplay-and-comparison-five-tribes
https://boardgamegeek.com/thread/1765727/subtle-elegant-pattern-spotting-

https://boardgamegeek.com/thread/2168648/family-docs-review-dice-hospital
https://boardgamegeek.com/thread/2292796/dice-hospital-review-if-theme-hospital-was-board-g
https://boardgamegeek.com/thread/2202748/board-geek-girl-reviews-dice-hospital
https://boardgamegeek.com/thread/2147242/medical-students-review-dice-hospital
https://boardgamegeek.com/thread/1708039/herbaceous-solo-review-perfect-solo-filler-almost
https://boardgamegeek.com/thread/1751554/20x-reviews-42-herbaceous-after-20-plays
https://boardgamegeek.com/thread/1838433/cardboard-clash-review-two-herbaceous
https://boardgamegeek.com/thread/2158775/solo-gaming-herbaceous-meeple-myself-and-i-review
https://boardgamegeek.com/thread/1821309/one-board-family-review-herbaceous
https://boardgamegeek.com/thread/160928/elfenland-review-14-easy-steps
https://boardgamegeek.com/thread/132649/elfenland-so-whats-not
https://boardgamegeek.com/thread/866401/super-fun-despite-theme
https://boardgamegeek.com/thread/144081/couples-review
http

In [19]:
df_found.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Data columns (total 6 columns):
id             49 non-null int64
name           49 non-null object
game           49 non-null object
url            49 non-null object
reviews        49 non-null object
description    49 non-null object
dtypes: int64(1), object(5)
memory usage: 2.7+ KB


In [20]:
df_found.to_csv('df_found_1305_19May', index=False, encoding='utf-8')

#### I had an issue where the function failed I think due to timeout of page reloading.
#### Since this scraping process took so long to run, I ran it in batches in a for loop.

In [280]:
for i in range(50):
    test_df = get_review_data(df4.iloc[20*i:20*i+19, :])
    test_df.to_csv('top_1000_reviews.csv', 'a', index=False)
    df6_test = df6_test.append(test_df)

https://boardgamegeek.com/thread/1747684/not-perfect-best-dungeon-crawler-despite-faults
https://boardgamegeek.com/thread/1859481/thoughts-after-finishing-game
https://boardgamegeek.com/thread/1760237/quick-procon-review-why-gloomhaven-ultimately-didn
https://boardgamegeek.com/thread/1963477/what-gloomhaven-gets-wrong
https://boardgamegeek.com/thread/1718321/gloomhaven-first-impressions
https://boardgamegeek.com/thread/1442849/glass-meeple-takes-peek-pandemic-legacy-no-spoiler
https://boardgamegeek.com/thread/1509235/pandemic-legacy-poor-successor-risk-legacy
https://boardgamegeek.com/thread/1498463/step-down-pandemic-avoid-plague
https://boardgamegeek.com/thread/1540066/polarizing-game-full-magic-let-down-few-key-failur
https://boardgamegeek.com/thread/1459040/glass-meeple-illuminates-pandemic-legacy-spoilers
https://boardgamegeek.com/thread/1644676/humanitys-quick-start-guide-terraforming-mars
https://boardgamegeek.com/thread/1618172/review-after-4-games
https://boardgamegeek.com/thr

https://boardgamegeek.com/thread/1501736/viticulture-essentials-edition-masterful-example-w
https://boardgamegeek.com/thread/1507731/viticulture-essential-edition-claustrophobic-worke
https://boardgamegeek.com/thread/1658930/odin-massive-massive-review-feast-odin-befits-such
https://boardgamegeek.com/thread/1656517/early-impressions-comparisons-uwes-previous-games
https://boardgamegeek.com/thread/1828388/after-50-plays-ranior-reviews-feast-odin
https://boardgamegeek.com/thread/1658399/stephs-initial-impressions-and-insights-uwes-newes
https://boardgamegeek.com/thread/1654855/first-impression-after-three-plays
https://boardgamegeek.com/thread/679445/enders-comprehensive-overview-analysis-puerto-rico
https://boardgamegeek.com/thread/649398/pictorial-overview-edition-puerto-rico-illustrated
https://boardgamegeek.com/thread/1336469/voices-experience-meta-analysis-puerto-rico
https://boardgamegeek.com/thread/221970/so-whats-not
https://boardgamegeek.com/thread/2182759/puerto-rico-detailed-r

https://boardgamegeek.com/thread/349388/pre-spiel-3-tired-player-shortened-game
https://boardgamegeek.com/thread/1313021/first-impression-what-game
https://boardgamegeek.com/thread/1376793/minty-fresh-or-bah-humbug
https://boardgamegeek.com/thread/1393813/purge-699-voyages-marco-polo-another-euro-game-abo
https://boardgamegeek.com/thread/1355437/jflartners-500-word-review-marco-polo
https://boardgamegeek.com/thread/1313022/ersteindruck-deutsch-was-fur-ein-spiel
https://boardgamegeek.com/thread/1033838/castaways-quick-start-guide-robinson-crusoe
https://boardgamegeek.com/thread/891338/robinson-crusoe-review-or-limiting-my-gilligans-is
https://boardgamegeek.com/thread/873409/europhile-reviews-first-play-new-essen-2012-releas
https://boardgamegeek.com/thread/995656/yep-youre-doomed-sessionreview-robinson-crusoe-adv
https://boardgamegeek.com/thread/888008/thank-you-game
https://boardgamegeek.com/thread/2119028/why-i-didnt-everdell
https://boardgamegeek.com/thread/2112933/everdell-space-bif

https://boardgamegeek.com/thread/1093109/eldritch-horror-proper-big-review
https://boardgamegeek.com/thread/1313071/12365478th-review-comparing-eldritch-horror-arkham
https://boardgamegeek.com/thread/1607010/preview-rise-beautiful-city-and-beautiful-game
https://boardgamegeek.com/thread/1683995/masterful-build-creaking-shelves-reviews-lisboa
https://boardgamegeek.com/thread/1822204/lisboa-board-game-spotlight-review
https://boardgamegeek.com/thread/1917528/lisboa-pros-and-cons-review
https://boardgamegeek.com/thread/1877396/how-learn-play-lisboa-and-how-teach-wife
https://boardgamegeek.com/thread/1479787/long-and-short-mombasa
https://boardgamegeek.com/thread/1465237/roelovichs-quick-point-point-review-mombasa
https://boardgamegeek.com/thread/1504782/minas-mini-review-mombasa-2
https://boardgamegeek.com/thread/1507359/cooking-books-scramble-africa
https://boardgamegeek.com/thread/1492741/mombasa-2-player-game
https://boardgamegeek.com/thread/419949/review-dominion-intrigue-angelgabriel

https://boardgamegeek.com/thread/1713625/broken-meeple-pandemic-iberia-review-theme-makes-c
https://boardgamegeek.com/thread/1776160/pandemic-iberia
https://boardgamegeek.com/thread/948451/kemet-jackal
https://boardgamegeek.com/thread/997872/i-love-it-fat-kid-love-cake-review-kemet
https://boardgamegeek.com/thread/1178789/what-doctor-ordered
https://boardgamegeek.com/thread/1094732/arwen-cat-reviews-kemet
https://boardgamegeek.com/thread/1430217/everything-sucks-about-kemet
https://boardgamegeek.com/thread/942443/why-it-works-review-after-wave-2
https://boardgamegeek.com/thread/857358/initial-impressionsreview
https://boardgamegeek.com/thread/918648/ten-things-and-five-things-dislike-about-star-wars
https://boardgamegeek.com/thread/851715/all-wings-report-x-wing-review-3-lengths
https://boardgamegeek.com/thread/865799/europhile-reviews-phantom-menace-or-empire-strikes
https://boardgamegeek.com/thread/1492891/minas-not-so-mini-review-grand-austria-hotel-2
https://boardgamegeek.com/threa

...

https://boardgamegeek.com/thread/1850492/meeple-overboard-review-i-choo-choo-choose-you-wth
https://boardgamegeek.com/thread/1852654/whistle-stop-review-jaysenheadleywrites
https://boardgamegeek.com/thread/1873559/he-said-she-said-whistle-stop-review
https://boardgamegeek.com/thread/1863280/boardgame-generations-whistle-stop
https://boardgamegeek.com/thread/2042151/whistle-stop-review
https://boardgamegeek.com/thread/1995785/fantastic-and-underlooked-feld
https://boardgamegeek.com/thread/1275842/bite-sized-review-la-isla-bggcon
https://boardgamegeek.com/thread/1257789/gotta-catch-em-all
https://boardgamegeek.com/thread/1288876/board-gamer-la-isla-overview-and-review
https://boardgamegeek.com/thread/1252308/quick-review-la-isla
https://boardgamegeek.com/thread/344839/video-review-ultimate-werewolf-ultimate-edition
https://boardgamegeek.com/thread/1077180/enders-comprehensive-pictorial-overview-ultimate-g
https://boardgamegeek.com/thread/703430/25-questions-about-ultimate-werewolf-ultima

https://boardgamegeek.com/thread/908891/europhile-reviews-stunning-classic-turn-millennium
https://boardgamegeek.com/thread/594587/stack-em-high-stack-em-low-world-java
https://boardgamegeek.com/thread/309278/will-my-friends-want-play-java
https://boardgamegeek.com/thread/156559/my-take-java-rice-terrace-themed
https://boardgamegeek.com/thread/1447875/minas-mini-review-new-york-1901-2
https://boardgamegeek.com/thread/1407039/terrific-family-board-game-board-game-family-revie
https://boardgamegeek.com/thread/1381328/clever-building-game-new-york
https://boardgamegeek.com/thread/1438571/review-new-york-1901
https://boardgamegeek.com/thread/1444030/purge-831-new-york-1901-tile-laying-turn-century-a
https://boardgamegeek.com/thread/635663/rally-good-fun
https://boardgamegeek.com/thread/579498/5-gas-gas
https://boardgamegeek.com/thread/615643/boardgames-blighty-reviews-rallyman
https://boardgamegeek.com/thread/592450/why-i-want-one-bonus-points-solo-play
https://boardgamegeek.com/thread/105

https://boardgamegeek.com/thread/2060892/agra-four-sided-game-review
https://boardgamegeek.com/thread/2076088/story-board-reviews-agra
https://boardgamegeek.com/thread/272964/quick-review-werewolves-millers-hollow
https://boardgamegeek.com/thread/548897/werewolves-where-shouting-fun
https://boardgamegeek.com/thread/629020/review-werewolves-millers-hollow
https://boardgamegeek.com/thread/1899160/resena-por-el-viernes-toca-jugar
https://boardgamegeek.com/thread/1466241/bring-your-poker-face-and-best-persuasion-skills
first_post is none
https://boardgamegeek.com/thread/2264794/luxor-detailed-review
https://boardgamegeek.com/thread/2226851/luxor-breakdown-collectors-edition-queenies
https://boardgamegeek.com/thread/2047781/family-game-core-game-mechanic
https://boardgamegeek.com/thread/2040339/luxor-great-light-weight-game
https://boardgamegeek.com/thread/2124545/enjoyable-race-through-tombs-pharaohs-players-coll
https://boardgamegeek.com/thread/322352/french-people-are-just-plain-weird
ht

https://boardgamegeek.com/thread/625383/father-geeks-review-evo
https://boardgamegeek.com/thread/922667/why-did-i-buy-carcassonne-winter-edition-boxed-set
https://boardgamegeek.com/thread/895703/winter-already-here-review-winter-edition-carcasso
https://boardgamegeek.com/thread/1914335/my-favorite-version-my-favorite-tile-laying-game
https://boardgamegeek.com/thread/1666147/meeple-meeple-review-carcassonne-winter-edition
https://boardgamegeek.com/thread/2026860/carcassonne-winter-review
https://boardgamegeek.com/thread/308740/fire-and-axe-and-math
https://boardgamegeek.com/thread/167447/review-fire-and-axe-viking-saga
https://boardgamegeek.com/thread/1633669/fire-axe-viking-saga-detailed-review
https://boardgamegeek.com/thread/270264/bookshelf-games-video-review
https://boardgamegeek.com/thread/404368/why-i-fire-and-axe-vikings-saga
https://boardgamegeek.com/thread/1202173/lonely-meeple-reviews-hostage-negotiator-back-or-m
https://boardgamegeek.com/thread/1368051/10-points-review-hosta

https://boardgamegeek.com/thread/1978333/ulm-review
https://boardgamegeek.com/thread/1694418/ulm-60-minutes-fun-and-frustration
https://boardgamegeek.com/thread/1665074/sparrows-and-relativity
https://boardgamegeek.com/thread/1362899/enders-comprehensive-pictorial-overview-new-editio
https://boardgamegeek.com/thread/506339/video-review-mystery-rummy-jack-ripper
https://boardgamegeek.com/thread/386287/original-mystery-rummy-still-best
https://boardgamegeek.com/thread/163457/you-dont-know-jack
https://boardgamegeek.com/thread/750119/review-one-our-most-played-games
https://boardgamegeek.com/thread/1481325/board-gamer-reviews-122-council-four
https://boardgamegeek.com/thread/1986868/islaythedragoncom-reviews-council-4
https://boardgamegeek.com/thread/1459680/council-four
https://boardgamegeek.com/thread/1723241/story-board-reviews-council-4
https://boardgamegeek.com/thread/176626/wow-lite-wagaming-anomaly
https://boardgamegeek.com/thread/198170/wings-war-essential-your-collection
https://

In [281]:
df6_test.head()

,id,game,url,description,reviews
0,174430,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,Gloomhaven is a game of Euro-inspired tactica...,[ Overall SummaryGloomhaven is truly an amazin...
1,161936,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,Pandemic Legacy is a co-operative campaign gam...,[ Pandemic: Legacy - Write Your Own Legacy in ...
2,167791,Terraforming Mars,https://boardgamegeek.com/boardgame/167791/ter...,"In the 2400s, mankind begins to terraform the ...",[ 1. There's only one thing I don't like about...
3,224517,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,Brass: Birmingham is an economic strategy game...,[ IntroductionI love the original Brass game. ...
4,182028,Through the Ages: A New Story of Civilization,https://boardgamegeek.com/boardgame/182028/thr...,Through the Ages: A New Story of Civilization ...,[ is the 2015 reimplementation of the popular...


In [282]:
df6_test.tail()

,id,game,url,description,reviews
994,228504,Cat Lady,https://boardgamegeek.com/boardgame/228504/cat...,"In Cat Lady, players are cat ladies, part of a...","[ Cats are cute and adorable, right? They shou..."
995,265402,In the Hall of the Mountain King,https://boardgamegeek.com/boardgame/265402/hal...,Play as trolls rebuilding your abandoned kingd...,[ In the Hall of the Mountain King by Burnt Is...
996,61692,Axis & Allies Europe 1940,https://boardgamegeek.com/boardgame/61692/axis...,"Axis &amp; Allies Europe 1940, a deluxe editio...","[ First of all, how can a review work when a g..."
997,180040,Villages of Valeria,https://boardgamegeek.com/boardgame/180040/vil...,A village-building card game set in the world ...,[ Disclaimer: This review of Villages of Valer...
998,176371,Explorers of the North Sea,https://boardgamegeek.com/boardgame/176371/exp...,Explorers of the North Sea is set in the latte...,[ The GameLike the other games of the ‘North S...


In [284]:
# df6_test.to_csv('top_950_14May.csv', index=False)

In [289]:
df6_test['reviews'][998]

[" The GameLike the other games of the ‘North Sea’ trilogy, Explorers of the North Sea is a game about Vikings! In this game, the players will placing tiles and moving their Viking longships in an effort to most efficiently collect livestock, raid settlements, attack enemy ships, and control islands. Let’s see how the game plays! The Board and ComponentsThe board is comprised of the starting tile (of which there are 2, double sided) and 48 game tiles, one of which will be placed at the start of each player turn. There are 26 livestock (3 cattle, 3 horses, 4 sheep, 5 goats, 6 chickens, 6 pigs). These will spawn on their respective tiles and will be collected by the players (i.e. there are only 3 tiles with a cow in the stack of 48).There are 14 settlement tokens. These are randomly placed when settlements enter the game board and indicate the strength of the settlement and how many points are gained if it is raided by a player.The game also includes 12 enemy ship tokens. These will also